In [1]:
from build_dataset import MRILesionDatasetBuilder, LatentImageProcessor

2025-05-12 16:38:56.684543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747060736.706170 1464378 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747060736.712694 1464378 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747060736.729142 1464378 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747060736.729161 1464378 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747060736.729164 1464378 computation_placer.cc:177] computation placer alr

# Build the dataset for the brain generation model
- Inpaint lesions with simple method in the slices with lesions
- Include SHIFTS and WMH2017 datasets

### VH

In [3]:
data_folder="/home/benet/data"
input_folder="VH"
output_folder="generation2D_VH-SHIFTS-WMH2017"
folders=["train", "test"]
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=1
seed = 17844
skip_empty_masks=False
fill_lesion=True

# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()


# output_folder="lesion2D_VH"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

output_folder="lesion2D_VH_split"
fill_lesion=False
train_split=0.7
dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 741, train examples: 507 (68.42%), test examples: 234 (31.58%)
In the hole dataset: Total examples: 741, train examples: 507 (68.42%), test examples: 234 (31.58%)


### Latent

In [ ]:
from pathlib import Path
import os
repo_path= Path.cwd().resolve()
while '.gitignore' not in os.listdir(repo_path): # while not in the root of the repo
    repo_path = repo_path.parent #go up one level
    print(repo_path)
resolution = 256
processor_train = LatentImageProcessor(repo_path, output_dir = "/home/benet/data/lesion2D_VH_split/train/latent", scale=True, 
                                 resolution=resolution, finetuned_vae=False, input_dir = "/home/benet/data/lesion2D_VH_split/train/flair")
processor_train.process_images()

processor_test = LatentImageProcessor(repo_path, output_dir = "/home/benet/data/lesion2D_VH_split/test/latent", scale=True,
                                    resolution=resolution, finetuned_vae=False, input_dir = "/home/benet/data/lesion2D_VH_split/test/flair")
processor_test.process_images()

/home/benet/tfg
Device: cuda
Found 507 images in the dataset folder
2028 latent images saved in /home/benet/data/lesion2D_VH_split/train/latent
Device: cuda
Found 234 images in the dataset folder
936 latent images saved in /home/benet/data/lesion2D_VH_split/test/latent


: 

### SHIFTS

In [6]:
data_folder="/home/benet/data"
input_folder="SHIFTS_preprocessedMNI" ###
output_folder="generation2D_VH-SHIFTS-WMH2017"
folders=["dev_in", "dev_out", "eval_in", "train"] ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=1
seed = 17844
skip_empty_masks=False
fill_lesion=True

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


# output_folder="lesion2D_SHIFTS"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 1274, train examples: 1274 (100.00%), test examples: 0 (0.00%)
In the hole dataset: Total examples: 2015, train examples: 2015 (100.00%), test examples: 0 (0.00%)


### WMH2017

In [7]:
data_folder="/home/benet/data"
input_folder="WMH2017_preprocessedMNI" ###
output_folder="generation2D_VH-SHIFTS-WMH2017"
folders=None ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=1
seed = 17844
skip_empty_masks=False
fill_lesion=True

dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
dataset_builder.build_dataset()


# output_folder="lesion2D_WMH2017"
# fill_lesion=False
# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks, fill_lesion)
# dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 780, train examples: 780 (100.00%), test examples: 0 (0.00%)
In the hole dataset: Total examples: 2795, train examples: 2795 (100.00%), test examples: 0 (0.00%)
